# academictwitteR

The release of the new [Twitter v2 API](https://developer.twitter.com/en/docs/twitter-api/early-access) with its [Academic Research product track](https://developer.twitter.com/en/products/twitter-api/academic-research) has sparked the creation of several new `R` packages: [`academictwitteR`](https://github.com/cjbarrie/academictwitteR), [`twitterAcademic`](https://github.com/kasperwelbers/twitterAcademic), [`twitteRacademic`](https://github.com/mittendo/twitteRacademic), and [`RTwitterV2`](https://github.com/MaelKubli/RTwitterV2). As the names already indicate, these packages are quite similar in the functionalities that they offer. From those options, [`academictwitteR`](https://github.com/cjbarrie/academictwitteR) has the best documentation and is the easiest one to use. It also seems to be the most actively developed one. Hence, we will focus on that package in this notebook. Another new `R` package that makes use of the v2 API is [`voson.tcn`](https://github.com/vosonlab/voson.tcn) (from [*VOSON Lab*](http://vosonlab.net/) who also created [`vosonSML`](https://github.com/vosonlab/vosonSML)). However, `voson.tcn` was created for analyzing conversation networks on Twitter, so the units of analyses are conversations instead of users for that package.

*Note*: The first version of the [`academictwitteR` package](https://github.com/cjbarrie/academictwitteR) was based on a [gist](https://gist.github.com/schochastics/1ff42c0211916d73fc98ba8ad0dcb261) by [David Schoch](https://gist.github.com/schochastics).

## Load packages

In addition to the `academictwitteR` package, we load the packages [`magrittr`](https://magrittr.tidyverse.org/index.html) and [`dplyr`](https://dplyr.tidyverse.org/) from the [`tidyverse`](https://www.tidyverse.org/) for some (minor) data wrangling.

In [ ]:
library(academictwitteR)
library(magrittr)
library(dplyr)

## Authentication

Before we can collect data via the Twitter v2 API, we need to set up our credentials. For this we need a [bearer token](https://developer.twitter.com/en/docs/authentication/oauth-2-0/bearer-tokens) for an app created for use with the v2 API (which you can get access to via the [academic research product track](https://developer.twitter.com/en/solutions/academic-research/products-for-researchers)).

**NB**: You should treat all information relating to your API access like a password and never share it or post it publicly anywhere. Although nobody except you should be able to access your personal instance of this notebook (and your edits will also not be persistent if you do not have/use a *GESIS Notebooks* user account), if you want to be extra cautious, you can delete your API access information from the following cell after running it once (and save the notebook again after that).

In [ ]:
# enter a bearer token for your v2 API app here
bearer_token <- ""

## Import list of accounts

The file [twitter_accounts.csv](./data/twitter_accounts.csv) in the `data` folder of this repository contains the Twitter screen names of [*GESIS - Leibniz Institute for the Social Sciences*](https://www.gesis.org/en/home) and the [*Social Data Science Lab*](http://socialdatalab.net/) at *Cardiff University* which we will use in the following examples.

In [ ]:
users_df <- read.csv("./data/twitter_accounts.csv")
users = as.character(users_df$Screen.Name)

## Search & collect tweets

We now collect all tweets sent from these accounts between April 1st and May 31st, 2021.

In [ ]:
tweets_df <- get_user_tweets(users,
                             "2021-04-01T00:00:00Z",
                             "2021-05-31T00:00:00Z",
                             bearer_token)

## User information

If we want to get some (additional) information about the accounts, we can use the `get_user_profiles` function.

In [ ]:
user_profiles <- get_user_profile(unique(tweets_df$author_id), bearer_token)

We can now combine the profile information with the tweets data. In the following cell, we just use the variables `name`, `username`, and `location` from the profile information. To see what else is included in that dataframe you could, e.g., run `names(user_profiles)` or `head(user_profiles)`.

In [ ]:
tweets_combined <- tweets_df %>% 
    left_join(select(user_profiles, id, name, username, location),
              by = c("author_id" = "id"))

## Process & save results

If we check the types of columns in the `tweets_combined` object, we can see that some of them are lists or dataframes.

In [ ]:
sapply(tweets_combined, class)

For example, the column `public_metrics` that contains information on how often a tweet has been liked, retweeted, quoted, or replied to is a dataframe.

In [ ]:
head(tweets_combined$public_metrics)

If we want to save our results as a `.csv` file using the base `R` function `write.csv()`, the object cannot include list- or dataframe-columns. We could split the list columns into "regular" vector columns (e.g., using the [`unnest()` function](https://tidyr.tidyverse.org/reference/nest.html) from the [`tidyr` package](https://tidyr.tidyverse.org/index.html)) and/or append the dataframe columns (using `bind_cols()` from `dplyr` or `cbind()` from base `R`). However, to keep it simple here, we will just remove all dataframe- and list-columns and so that we can save the resulting object in a `.csv` file.

In [ ]:
tweets_combined_df <- tweets_combined %>% 
    select(where(~ !is.data.frame(.x) & !is.list(.x)))

Before we save the resulting dataframe as `csv`, we can check it to make sure that it does not contain any list- or dataframe-columns anymore.

In [ ]:
str(tweets_combined_df)
head(tweets_combined_df)

In [ ]:
write.csv(tweets_combined_df, "./data/tweets_apiv2_r.csv")